In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector
from matplotlib.patches import Rectangle
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from PIL import Image, ImageTk
import os

class MorphologicalOperationsApp:
    def __init__(self):
        self.original_image = None
        self.current_image = None
        self.roi_coords = None
        self.kernel_size = 5
        self.selected_operation = None
        
        self.root = tk.Tk()
        self.root.title("Morphological Operations - Practical Assessment")
        self.root.geometry("1200x800")
        
        self.setup_gui()
        
        self.load_default_image()
        
    def setup_gui(self):
        # Main frame
        main_frame = ttk.Frame(self.root)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Control panel
        control_frame = ttk.LabelFrame(main_frame, text="Controls", padding=10)
        control_frame.pack(side=tk.TOP, fill=tk.X, pady=(0, 10))
        
        # Load image button
        ttk.Button(control_frame, text="Load Image", command=self.load_image).pack(side=tk.LEFT, padx=(0, 10))
        
        # Kernel size control
        ttk.Label(control_frame, text="Kernel Size:").pack(side=tk.LEFT, padx=(10, 5))
        self.kernel_var = tk.IntVar(value=5)
        kernel_spinbox = ttk.Spinbox(control_frame, from_=3, to=15, width=5, textvariable=self.kernel_var, increment=2)
        kernel_spinbox.pack(side=tk.LEFT, padx=(0, 10))
        
        # Reset button
        ttk.Button(control_frame, text="Reset", command=self.reset_image).pack(side=tk.LEFT, padx=(10, 0))
        
        # Operations frame
        ops_frame = ttk.LabelFrame(main_frame, text="Morphological Operations (Keyboard Shortcuts)", padding=10)
        ops_frame.pack(side=tk.TOP, fill=tk.X, pady=(0, 10))
        
        # Operation buttons with keyboard shortcuts
        operations = [
            ("Dilation (d)", "dilation", "d"),
            ("Erosion (e)", "erosion", "e"),
            ("Opening (o)", "opening", "o"),
            ("Closing (c)", "closing", "c"),
            ("Morphological Gradient (g)", "gradient", "g"),
            ("Top Hat (t)", "tophat", "t"),
            ("Black Hat (b)", "blackhat", "b")
        ]
        
        for i, (text, op, key) in enumerate(operations):
            btn = ttk.Button(ops_frame, text=text, command=lambda o=op: self.apply_operation(o))
            btn.pack(side=tk.LEFT, padx=5)
            # Bind keyboard shortcuts
            self.root.bind(f"<KeyPress-{key}>", lambda e, o=op: self.apply_operation(o))
        
        # Image display frame
        image_frame = ttk.Frame(main_frame)
        image_frame.pack(side=tk.BOTTOM, fill=tk.BOTH, expand=True)
        
        # Original image frame
        original_frame = ttk.LabelFrame(image_frame, text="Original Image", padding=5)
        original_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=(0, 5))
        
        self.original_canvas = tk.Canvas(original_frame, bg="white")
        self.original_canvas.pack(fill=tk.BOTH, expand=True)
        
        # Processed image frame
        processed_frame = ttk.LabelFrame(image_frame, text="Processed Image", padding=5)
        processed_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True, padx=(5, 0))
        
        self.processed_canvas = tk.Canvas(processed_frame, bg="white")
        self.processed_canvas.pack(fill=tk.BOTH, expand=True)
        
        # Bind mouse events for ROI selection on original image
        self.original_canvas.bind("<Button-1>", self.start_roi_selection)
        self.original_canvas.bind("<B1-Motion>", self.update_roi_selection)
        self.original_canvas.bind("<ButtonRelease-1>", self.end_roi_selection)
        
        # Focus on root for keyboard events
        self.root.focus_set()
        
        # ROI selection variables
        self.roi_start = None
        self.roi_rect = None
        
    def load_default_image(self):
        """No default image - user must load an image"""
        self.original_image = None
        self.current_image = None
        
    def load_image(self):
        """Load image from file"""
        file_path = filedialog.askopenfilename(
            title="Select Image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png *.bmp *.tiff *.tif")]
        )
        
        if file_path:
            img = cv2.imread(file_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                self.original_image = img
                self.current_image = img.copy()
                self.roi_coords = None
                self.display_image()
            else:
                messagebox.showerror("Error", "Could not load the selected image.")
    
    def display_image(self):
        """Display original and current images side by side"""
        if self.original_image is None:
            return
            
        # Display original image
        self.display_on_canvas(self.original_image, self.original_canvas, "original")
        
        # Display current/processed image
        if self.current_image is not None:
            self.display_on_canvas(self.current_image, self.processed_canvas, "processed")
            
    def display_on_canvas(self, image, canvas, image_type):
        """Helper method to display image on specified canvas"""
        # Convert to PIL Image
        pil_image = Image.fromarray(image)
        
        # Resize if too large
        canvas_width = canvas.winfo_width()
        canvas_height = canvas.winfo_height()
        
        if canvas_width > 1 and canvas_height > 1:
            img_width, img_height = pil_image.size
            scale = min(canvas_width / img_width, canvas_height / img_height, 1.0)
            
            if scale < 1.0:
                new_width = int(img_width * scale)
                new_height = int(img_height * scale)
                pil_image = pil_image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
        # Convert to PhotoImage
        if image_type == "original":
            self.original_photo = ImageTk.PhotoImage(pil_image)
            # Clear canvas and display image
            canvas.delete("all")
            canvas.create_image(0, 0, anchor=tk.NW, image=self.original_photo)
        else:
            self.processed_photo = ImageTk.PhotoImage(pil_image)
            # Clear canvas and display image
            canvas.delete("all")
            canvas.create_image(0, 0, anchor=tk.NW, image=self.processed_photo)
        
    def start_roi_selection(self, event):
        """Start ROI selection"""
        self.roi_start = (self.original_canvas.canvasx(event.x), self.original_canvas.canvasy(event.y))
        if self.roi_rect:
            self.original_canvas.delete(self.roi_rect)
            
    def update_roi_selection(self, event):
        """Update ROI selection rectangle"""
        if self.roi_start:
            current_pos = (self.original_canvas.canvasx(event.x), self.original_canvas.canvasy(event.y))
            
            if self.roi_rect:
                self.original_canvas.delete(self.roi_rect)
                
            self.roi_rect = self.original_canvas.create_rectangle(
                self.roi_start[0], self.roi_start[1],
                current_pos[0], current_pos[1],
                outline="red", width=2
            )
            
    def end_roi_selection(self, event):
        """End ROI selection and store coordinates"""
        if self.roi_start:
            end_pos = (self.original_canvas.canvasx(event.x), self.original_canvas.canvasy(event.y))
            
            # Calculate ROI coordinates in image space
            img_height, img_width = self.original_image.shape[:2]
            
            # Get the scale factor
            if hasattr(self, 'original_photo'):
                photo_width = self.original_photo.width()
                photo_height = self.original_photo.height()
                scale_x = img_width / photo_width
                scale_y = img_height / photo_height
            else:
                scale_x = scale_y = 1.0
            
            x1 = max(0, min(int(self.roi_start[0] * scale_x), int(end_pos[0] * scale_x)))
            x2 = max(0, max(int(self.roi_start[0] * scale_x), int(end_pos[0] * scale_x)))
            y1 = max(0, min(int(self.roi_start[1] * scale_y), int(end_pos[1] * scale_y)))
            y2 = max(0, max(int(self.roi_start[1] * scale_y), int(end_pos[1] * scale_y)))
            
            # Ensure coordinates are within image bounds
            x1 = max(0, min(x1, img_width))
            x2 = max(0, min(x2, img_width))
            y1 = max(0, min(y1, img_height))
            y2 = max(0, min(y2, img_height))
            
            if x2 > x1 and y2 > y1:
                self.roi_coords = (x1, y1, x2, y2)
                print(f"ROI selected: ({x1}, {y1}) to ({x2}, {y2})")
            else:
                self.roi_coords = None
                print("Invalid ROI selection")
                
    def apply_operation(self, operation):
        """Apply morphological operation"""
        if self.current_image is None:
            messagebox.showwarning("Warning", "Please load an image first.")
            return
            
        if self.roi_coords is None:
            messagebox.showwarning("Warning", "Please select a Region of Interest (ROI) first by clicking and dragging on the image.")
            return
            
        self.selected_operation = operation
        kernel_size = self.kernel_var.get()
        
        # Ensure odd kernel size
        if kernel_size % 2 == 0:
            kernel_size += 1
            
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
        
        # Get ROI
        x1, y1, x2, y2 = self.roi_coords
        roi = self.current_image[y1:y2, x1:x2].copy()
        
        # Convert to grayscale for morphological operations
        if len(roi.shape) == 3:
            roi_gray = cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
        else:
            roi_gray = roi
            
        # Convert to binary image for better morphological results
        _, roi_binary = cv2.threshold(roi_gray, 127, 255, cv2.THRESH_BINARY)
            
        # Apply morphological operation
        if operation == "dilation":
            result = cv2.dilate(roi_binary, kernel, iterations=1)
        elif operation == "erosion":
            result = cv2.erode(roi_binary, kernel, iterations=1)
        elif operation == "opening":
            # Opening = Erosion followed by Dilation (removes noise, smooths)
            result = cv2.morphologyEx(roi_binary, cv2.MORPH_OPEN, kernel, iterations=2)
        elif operation == "closing":
            # Closing = Dilation followed by Erosion (fills gaps, connects)
            result = cv2.morphologyEx(roi_binary, cv2.MORPH_CLOSE, kernel, iterations=2)
        elif operation == "gradient":
            result = cv2.morphologyEx(roi_binary, cv2.MORPH_GRADIENT, kernel)
        elif operation == "tophat":
            result = cv2.morphologyEx(roi_binary, cv2.MORPH_TOPHAT, kernel)
        elif operation == "blackhat":
            result = cv2.morphologyEx(roi_binary, cv2.MORPH_BLACKHAT, kernel)
        else:
            return
            
        # Convert back to RGB if original was RGB
        if len(self.current_image.shape) == 3:
            result_rgb = cv2.cvtColor(result, cv2.COLOR_GRAY2RGB)
        else:
            result_rgb = result
            
        # Update the ROI in the current image
        self.current_image[y1:y2, x1:x2] = result_rgb
        
        # Display updated image
        self.display_image()
        
        # Redraw ROI rectangle on original image to keep it visible
        if self.roi_coords and hasattr(self, 'original_photo'):
            self.redraw_roi_rectangle()
        
        # Show operation info
        print(f"Applied {operation.title()} with kernel size {kernel_size} to ROI ({x1}, {y1}) to ({x2}, {y2})")
        print(f"Original ROI shape: {roi.shape}, Result shape: {result_rgb.shape}")
        print(f"Result min/max values: {result_rgb.min()}/{result_rgb.max()}")
        
    def redraw_roi_rectangle(self):
        """Redraw the ROI rectangle on the original image"""
        if self.roi_coords and hasattr(self, 'original_photo'):
            x1, y1, x2, y2 = self.roi_coords
            img_height, img_width = self.original_image.shape[:2]
            
            # Convert image coordinates back to canvas coordinates
            photo_width = self.original_photo.width()
            photo_height = self.original_photo.height()
            scale_x = photo_width / img_width
            scale_y = photo_height / img_height
            
            canvas_x1 = x1 * scale_x
            canvas_y1 = y1 * scale_y
            canvas_x2 = x2 * scale_x
            canvas_y2 = y2 * scale_y
            
            # Draw rectangle on original canvas
            self.roi_rect = self.original_canvas.create_rectangle(
                canvas_x1, canvas_y1, canvas_x2, canvas_y2,
                outline="red", width=2
            )
        
    def reset_image(self):
        """Reset to original image"""
        if self.original_image is not None:
            self.current_image = self.original_image.copy()
            self.roi_coords = None
            if self.roi_rect:
                self.original_canvas.delete(self.roi_rect)
                self.roi_rect = None
            self.display_image()
            print("Image reset to original")
            
    def run(self):
        """Start the application"""
        # Bind additional keyboard events
        self.root.bind("<KeyPress-r>", lambda e: self.reset_image())
        self.root.bind("<Escape>", lambda e: self.root.quit())
        
        print("Morphological Operations Application Started")
        print("Keyboard shortcuts:")
        print("d - Dilation")
        print("e - Erosion") 
        print("o - Opening")
        print("c - Closing")
        print("g - Morphological Gradient")
        print("t - Top Hat")
        print("b - Black Hat")
        print("r - Reset image")
        print("Esc - Quit application")
        print("\nInstructions:")
        print("1. Click and drag to select ROI")
        print("2. Press keyboard shortcut or click button to apply operation")
        print("3. Adjust kernel size for different effects")
        
        self.root.mainloop()

# Create and run the application
app = MorphologicalOperationsApp()
app.run()

Morphological Operations Application Started
Keyboard shortcuts:
d - Dilation
e - Erosion
o - Opening
c - Closing
g - Morphological Gradient
t - Top Hat
b - Black Hat
r - Reset image
Esc - Quit application

Instructions:
1. Click and drag to select ROI
2. Press keyboard shortcut or click button to apply operation
3. Adjust kernel size for different effects
ROI selected: (108, 84) to (211, 166)
ROI selected: (108, 84) to (211, 166)
Applied Tophat with kernel size 5 to ROI (108, 84) to (211, 166)
Original ROI shape: (82, 103, 3), Result shape: (82, 103, 3)
Result min/max values: 0/255
Applied Tophat with kernel size 5 to ROI (108, 84) to (211, 166)
Original ROI shape: (82, 103, 3), Result shape: (82, 103, 3)
Result min/max values: 0/255
Image reset to original
Image reset to original
ROI selected: (87, 66) to (262, 175)
ROI selected: (87, 66) to (262, 175)
Applied Blackhat with kernel size 5 to ROI (87, 66) to (262, 175)
Original ROI shape: (109, 175, 3), Result shape: (109, 175, 3)
Resu